<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/03/hw_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget -O bank+marketing.zip wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
!unzip bank+marketing.zip && unzip bank.zip 'bank-full.csv' -d .

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score, accuracy_score

In [3]:
df_path = "bank-full.csv"
df = pd.read_csv(df_path, sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df.shape

(45211, 17)

In [5]:
df = df[['age', 'job', 'marital', 'education', 'balance',
         'housing', 'contact', 'day', 'month', 'duration',
         'campaign', 'pdays', 'previous', 'poutcome', 'y']]
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [6]:
df.dtypes

,0
age,int64
job,object
marital,object
education,object
balance,int64
housing,object
contact,object
day,int64
month,object
duration,int64


In [7]:
df.isnull().sum() # no missing values

,0
age,0
job,0
marital,0
education,0
balance,0
housing,0
contact,0
day,0
month,0
duration,0


In [8]:
df.education.mode().iloc[0] # Q1

'secondary'

In [9]:
corr_matrix = df.iloc[:, :-1].corr(numeric_only=True)
corr_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [10]:
corr_mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
corr_matrix_upper_tri = corr_matrix.abs().where(corr_mask)
max_corr_features = corr_matrix_upper_tri.stack().idxmax()
max_corr = corr_matrix_upper_tri.stack().max()
max_corr_features, max_corr # Q2

(('pdays', 'previous'), 0.4548196354805043)

In [11]:
df.iloc[:, -1].value_counts() # check distinct labels

,count
y,
no,39922
yes,5289


In [12]:
# replace the values yes/no with 1/0
# https://stackoverflow.com/questions/52395179/change-dtype-of-specific-columns-with-iloc
df[df.columns[-1]] = df[df.columns[-1]].map({"yes": 1, "no": 0}).astype(int)
df.iloc[:, -1]

,y
0,0
1,0
2,0
3,0
4,0
...,...
45206,1
45207,1
45208,1
45209,0


In [13]:
def data_split(df, random_state):
    df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=random_state)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=random_state)
    return df_train, df_val, df_test

In [14]:
df_train, df_val, df_test = data_split(df, random_state=42)

In [15]:
cat_cols = df.iloc[:, :-1].select_dtypes(include=['object']).columns
mi_scores = []
for col in cat_cols:
    score = mutual_info_score(df_train[col], df_train.iloc[:, -1])
    mi_scores.append(score)

cat_cols[np.argmax(mi_scores)], dict(zip(cat_cols, mi_scores)) # Q3

('poutcome',
 {'job': 0.007316082778474635,
  'marital': 0.0020495925927810216,
  'education': 0.0026967549991295282,
  'housing': 0.010343105891750026,
  'contact': 0.013356062198247219,
  'month': 0.025090033443650246,
  'poutcome': 0.029532821290436224})

In [16]:
def one_hot_encode(df, dv, train):
    feature_dict = df.iloc[:, :-1].to_dict(orient='records')
    if train == True:
        X = dv.fit_transform(feature_dict)
    else:
        X = dv.transform(feature_dict)
    return X

In [17]:
def logistic_regression_fit(df, C, dv):
    X = one_hot_encode(df, dv, train=True)
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X, df.iloc[:, -1].values)
    return model

In [18]:
def logistic_regression_predict(model, df, dv):
    X = one_hot_encode(df, dv, train=False)
    y_pred = model.predict(X)
    return accuracy_score(df.iloc[:, -1].values, y_pred)

In [19]:
dv = DictVectorizer(sparse=False)
model = logistic_regression_fit(df_train, C=1.0, dv=dv)
acc_full = logistic_regression_predict(model, df_val, dv=dv)
round(acc_full, 2) # Q4

0.9

In [21]:
features = df_train.iloc[:, :-1].columns
cols = ['age', 'balance', 'marital', 'previous']
accuracies_diff = []
for col in features:
    dv = DictVectorizer(sparse=False)
    model = logistic_regression_fit(df_train.drop([col], axis=1), C=1.0, dv=dv)
    acc = logistic_regression_predict(model, df_val.drop([col], axis=1), dv=dv)
    accuracies_diff.append(acc_full - acc)

accuracies_diff_dict = dict(zip(features, accuracies_diff))
sub_accuracies_diff_dict = {k: v for k, v in accuracies_diff_dict.items() if k in cols}
min(sub_accuracies_diff_dict, key=sub_accuracies_diff_dict.get), sub_accuracies_diff_dict, accuracies_diff_dict # Q5

('age',
 {'age': -0.00044238000442387015,
  'marital': 0.0,
  'balance': -0.0001105950011059953,
  'previous': 0.0},
 {'age': -0.00044238000442387015,
  'job': -0.0002211900022119906,
  'marital': 0.0,
  'education': 0.0,
  'balance': -0.0001105950011059953,
  'housing': -0.0002211900022119906,
  'contact': 0.00044238000442375913,
  'day': -0.00044238000442387015,
  'month': 0.0011059500110593978,
  'duration': 0.011170095111700862,
  'campaign': 0.0006635700066356387,
  'pdays': 0.0,
  'previous': 0.0,
  'poutcome': 0.007520460075204571})

In [22]:
accuracies_r = []
C_list = [0.01, 0.1, 1, 10, 100]
for C in C_list:
    dv = DictVectorizer(sparse=False)
    model = logistic_regression_fit(df_train, C=C, dv=dv)
    acc = logistic_regression_predict(model, df_val, dv=dv)
    accuracies_r.append(round(acc, 3))

C_list[np.argmax(accuracies_r)], dict(zip(C_list, accuracies_r)) # Q6

(0.1, {0.01: 0.898, 0.1: 0.901, 1: 0.901, 10: 0.901, 100: 0.901})